In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit_ibm_runtime import (QiskitRuntimeService,Session,EstimatorV2,EstimatorOptions)



In [16]:
service = QiskitRuntimeService(channel="ibm_quantum",  # ibm_cloud 
                               token = '8a3e35ba3172edac10b0179786cb911eb5368c636c1c4542d158e3d17dabb33b75b7c7069d4fa4919f3bbb758622444d139f28356c32f337f18397b08559b99d')
QiskitRuntimeService.save_account(channel='ibm_quantum', overwrite=True,
                                  token = '8a3e35ba3172edac10b0179786cb911eb5368c636c1c4542d158e3d17dabb33b75b7c7069d4fa4919f3bbb758622444d139f28356c32f337f18397b08559b99d')
backend = QiskitRuntimeService().least_busy(simulator=False, operational=True, min_num_qubits=100)
backend.num_qubits

127

In [ ]:
def generate_alice_data(n_qubits):
    bits = np.random.randint(2, size=n_qubits)
    bases = np.random.randint(2, size=n_qubits)
    return bits, bases

def generate_bob_bases(n_qubits):
    return np.random.randint(2, size=n_qubits)

def create_single_bb84_circuit(alice_bit, alice_basis):
    qc = QuantumCircuit(1)
    if alice_bit == 1:
        qc.x(0)
    if alice_basis == 1:
        qc.h(0)
    return qc

def get_observable_for_bob_basis(bob_basis):
    return Pauli('Z') if bob_basis == 0 else Pauli('X')

def build_bb84_circuits_and_observables(n_qubits, alice_bits, alice_bases, bob_bases):

    circuits = []
    observables = []
    for i in range(n_qubits):
        qc = create_single_bb84_circuit(alice_bits[i], alice_bases[i])
        circuits.append(qc)

        obs = get_observable_for_bob_basis(bob_bases[i])
        observables.append(obs)

    return circuits, observables

def decode_estimator_values(values):
    measured_bits = []
    for val in values:
        if val >= 0:   # near +1
            measured_bits.append(0)
        else:          # near -1
            measured_bits.append(1)
    return measured_bits

def compute_qber(alice_bits, alice_bases, bob_bits, bob_bases):
    matching_indices = [i for i in range(len(alice_bits)) if alice_bases[i] == bob_bases[i]]
    if not matching_indices:
        return None

    mismatches = sum(alice_bits[i] != bob_bits[i] for i in matching_indices)
    return mismatches / len(matching_indices)
def print_matching_bases_and_keys(alice_bits, alice_bases, bob_bits, bob_bases):
    print("Index | Alice Basis | Bob Basis | Matching? | Alice Bit | Bob Bit")
    print("----------------------------------------------------------------------")
    for i in range(len(alice_bits)):
        match = alice_bases[i] == bob_bases[i]
        print(f"{i:5d} |     {alice_bases[i]}       |     {bob_bases[i]}     |   {'Yes' if match else ' No'}    |     {alice_bits[i]}     |    {bob_bits[i]}")

    # Build shared raw key from matched bases
    shared_indices = [i for i in range(len(alice_bits)) if alice_bases[i] == bob_bases[i]]
    alice_key = [alice_bits[i] for i in shared_indices]
    bob_key   = [bob_bits[i]   for i in shared_indices]

    print("\n✅ Shared Key (Same Bases Only):")
    print(f"Alice Key: {alice_key}")
    print([int(bit) for bit in alice_key])
    print(f"  Bob Key: {bob_key}")
    print(f"Lenght of Key: {len(bob_key)}")


In [ ]:
def run_bb84_local(n_qubits=10, shots=1024):
    alice_bits, alice_bases = generate_alice_data(n_qubits)
    bob_bases = generate_bob_bases(n_qubits)

    circuits, observables = build_bb84_circuits_and_observables(
        n_qubits, alice_bits, alice_bases, bob_bases
    )

    estimator = AerEstimator()
    job = estimator.run(circuits=circuits, observables=observables, shots=shots)
    result = job.result()

    values = result.values  
    bob_bits = decode_estimator_values(values)
    qber = compute_qber(alice_bits, alice_bases, bob_bits, bob_bases)
    print("\n[BB84 Local Simulation]")
    print(f"QBER = {qber}")
    print_matching_bases_and_keys(alice_bits, alice_bases, bob_bits, bob_bases)
    return qber


In [ ]:
def run_bb84_hardware(n_qubits=10):
    print(f"\n[BB84 on Real Hardware: {backend.name}]")

    alice_bits, alice_bases = generate_alice_data(n_qubits)
    bob_bases = generate_bob_bases(n_qubits)

    circuits, _ = build_bb84_circuits_and_observables(n_qubits, alice_bits, alice_bases, bob_bases)

    pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)
    transpiled_circuits = [pass_manager.run(circ) for circ in circuits]

    observables = []
    for qc, bob_basis in zip(transpiled_circuits, bob_bases):
        # Find which physical qubit was used
        q_idx = qc.find_bit(qc.qubits[0]).index
        label = ["I"] * qc.num_qubits
        label[q_idx] = "Z" if bob_basis == 0 else "X"
        observables.append(SparsePauliOp(Pauli("".join(label))))

    options = EstimatorOptions(
        resilience_level=1,
        dynamical_decoupling={"enable": True, "sequence_type": "XY4"}
    )

    with Session(backend=backend) as session:
        estimator = EstimatorV2(options=options)
        job = estimator.run(list(zip(transpiled_circuits, observables)))
        result = job.result()

    values = [float(r.data.evs) for r in result]
    bob_bits = decode_estimator_values(values)

    qber = compute_qber(alice_bits, alice_bases, bob_bits, bob_bases)
    print(f"QBER on {backend.name} = {qber}")
    print_matching_bases_and_keys(alice_bits, alice_bases, bob_bits, bob_bases)

    return qber


In [ ]:
if __name__ == "__main__":
    qber_local = run_bb84_local(n_qubits=1000, shots=1024)


C:\Users\utkar\AppData\Local\Temp\ipykernel_10964\2893317304.py:6: DeprecationWarning: Estimator has been deprecated as of Aer 0.15, please use EstimatorV2 instead.
  qber_local = run_bb84_local(n_qubits=1000, shots=1024)
C:\Users\utkar\AppData\Local\Temp\ipykernel_10964\2893317304.py:6: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstimator from qiskit.primitives.
  qber_local = run_bb84_local(n_qubits=1000, shots=1024)



[BB84 Local Simulation]
QBER = 0.0
Index | Alice Basis | Bob Basis | Matching? | Alice Bit | Bob Bit
----------------------------------------------------------------------
    0 |     1       |     0     |    No    |     0     |    0
    1 |     1       |     1     |   Yes    |     1     |    1
    2 |     1       |     1     |   Yes    |     1     |    1
    3 |     1       |     0     |    No    |     0     |    0
    4 |     1       |     0     |    No    |     1     |    1
    5 |     0       |     1     |    No    |     0     |    0
    6 |     0       |     0     |   Yes    |     0     |    0
    7 |     1       |     0     |    No    |     1     |    1
    8 |     0       |     0     |   Yes    |     1     |    1
    9 |     0       |     0     |   Yes    |     1     |    1
   10 |     1       |     1     |   Yes    |     0     |    0
   11 |     1       |     0     |    No    |     0     |    0
   12 |     1       |     1     |   Yes    |     0     |    0
   13 |     0       |

In [ ]:
if __name__ == "__main__":
    qber_hw = run_bb84_hardware(n_qubits=100)


[BB84 on Real Hardware: ibm_kyiv]


c:\Users\utkar\anaconda3\envs\myenv\lib\site-packages\qiskit_ibm_runtime\qiskit_runtime_service.py:857: UserWarning: Your current pending jobs are estimated to consume 179.65759633970868 quantum seconds, but you only have 149 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)


QBER on ibm_kyiv = 0.5454545454545454
Index | Alice Basis | Bob Basis | Matching? | Alice Bit | Bob Bit
----------------------------------------------------------------------
    0 |     1       |     1     |   Yes    |     1     |    0
    1 |     1       |     0     |    No    |     1     |    0
    2 |     1       |     1     |   Yes    |     0     |    0
    3 |     1       |     1     |   Yes    |     0     |    1
    4 |     1       |     1     |   Yes    |     1     |    0
    5 |     0       |     1     |    No    |     1     |    1
    6 |     0       |     1     |    No    |     1     |    0
    7 |     0       |     0     |   Yes    |     1     |    0
    8 |     0       |     1     |    No    |     1     |    1
    9 |     1       |     1     |   Yes    |     0     |    1
   10 |     1       |     1     |   Yes    |     1     |    0
   11 |     1       |     1     |   Yes    |     1     |    1
   12 |     1       |     1     |   Yes    |     0     |    1
   13 |     0      